In [126]:
# Okoneshnikova Eleonora, 932028

import pandas
import numpy
import openpyxl
import matplotlib.style
import matplotlib.pyplot as plt
import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

Climate_Region_Pub_labels = ['B', 'B', 'B', 'B', 'A']; # last digit of my passport's number is 4, then: 4 -> 'A', 0,1,2,3 -> 'B'
TEST_SIZE = 0.2

def replace_Climate_Region_Pub_to_A_B(dataFrame):
    for i in range(0, 5):
        dataFrame['Climate_Region_Pub'].replace(i+1, Climate_Region_Pub_labels[i], inplace=True)

def read_excel(filename, nrows):
    # Parameter `read_only=True` leads to excel rows only being loaded as-needed
    book = openpyxl.load_workbook(filename=filename, read_only=True, data_only=True)
    first_sheet = book.worksheets[0]
    rows_generator = first_sheet.values
    header_row = next(rows_generator)
    data_rows = [row for (_, row) in zip(range(nrows - 1), rows_generator)]
    data = pandas.DataFrame(data_rows, columns=header_row)
    replace_Climate_Region_Pub_to_A_B(data)
    return data

def get_rows_count_excel_file(filename):
    book = openpyxl.load_workbook(filename=filename, read_only=True, data_only=True)
    sheet = book.worksheets[0]
    return sheet.max_row

def get_bad_indexes(dataFrame):
    res_array = []
    for key, col in dataFrame.iteritems():
        index = 0
        for cell in col.values:
            if(isinstance(cell, datetime.datetime) or pandas.isnull(cell) or pandas.isna(cell)):
                if not index in res_array:
                    res_array.append(index)
            index += 1
    return res_array

rows_num = get_rows_count_excel_file('Houses.xlsx') # count rows in excel file
X_houses_data = read_excel('Houses.xlsx', rows_num) # load rows from excel file
### !!! df = df[df.line_race.notnull()] !!!
bad_indexes = get_bad_indexes(X_houses_data) # get numbers of rows with null/NA/datetime cells
X_houses_data.drop(bad_indexes, inplace=True) # drop rows with null/NA/datetime cells
X_houses_data.drop(0, inplace=True)

dummy_metromicro = pandas.get_dummies(X_houses_data['METROMICRO'])
dummy_ur = pandas.get_dummies(X_houses_data['UR'])

Y_houses_data = X_houses_data['Climate_Region_Pub']

X_houses_data = X_houses_data.drop(columns=['Climate_Region_Pub', 'METROMICRO', 'UR'])
X_houses_data = pandas.concat([X_houses_data, dummy_metromicro, dummy_ur], axis=1, sort=False)

X_houses_data = pandas.concat([X_houses_data['CDD30YR'],         # 20 most valuable features
                               X_houses_data['DIVISION'], 
                               X_houses_data['CDD65'],
                               X_houses_data['HDD65'],
                               X_houses_data['REPORTABLE_DOMAIN'],
                               X_houses_data['HDD30YR'],
                               X_houses_data['REGIONC'],
                               X_houses_data['SWAMPCOL'],
                               X_houses_data['AIA_Zone'],
                               X_houses_data['DOLELCOL'],
                               X_houses_data['TOTALBTUCOL'],
                               X_houses_data['TOTALDOLCOL'],
                               X_houses_data['DIPSTICK'],
                               X_houses_data['BTUELCOL'],
                               X_houses_data['NWEIGHT'],
                               X_houses_data['COOLTYPE'],
                               X_houses_data['DNTAC'],
                               X_houses_data['KWHCOL'],
                               X_houses_data['TOTALDOLSPH'],
                               X_houses_data['TOTALBTUSPH']], axis=1, sort=False)

X_houses_train, X_houses_test, y_houses_train, y_houses_test = train_test_split(X_houses_data, Y_houses_data, test_size=TEST_SIZE, random_state=0)
scaler = StandardScaler()
X_houses_train_scaled = scaler.fit_transform(X_houses_train)
X_houses_test_scaled = scaler.fit_transform(X_houses_test)

In [ ]:
"""
Houses importances:

 1) CDD30YR                        0.100389
 2) DIVISION                       0.055305
 3) CDD65                          0.050063
 4) HDD65                          0.047104
 5) REPORTABLE_DOMAIN              0.046672
 6) HDD30YR                        0.039484
 7) REGIONC                        0.020670
 8) SWAMPCOL                       0.019349
 9) AIA_Zone                       0.016494
10) DOLELCOL                       0.016024
11) TOTALBTUCOL                    0.014669
12) TOTALDOLCOL                    0.014415
13) DIPSTICK                       0.012927
14) BTUELCOL                       0.011932
15) NWEIGHT                        0.009750
16) COOLTYPE                       0.009167
17) DNTAC                          0.007913
18) KWHCOL                         0.007805
19) TOTALDOLSPH                    0.007230
20) TOTALBTUSPH                    0.007224
21) TOTCSQFT                       0.006298
22) AIRCOND                        0.005619
23) BTUELSPH                       0.004507
24) TOTUCSQFT                      0.004367
25) TOTALDOL                       0.004350
26) TEMPHOME                       0.004193
27) TEMPGONE                       0.004123
28) DOLELRFG                       0.004105
29) NUMCFAN                        0.004058
30) TEMPNITE                       0.004014
31) TOTALDOLOTH                    0.003938
32) TOTALBTU                       0.003841
33) WALLTYPE                       0.003841
34) MONEYPY                        0.003809
35) BTUEL                          0.003776
36) DOLLAREL                       0.003724
37) DOLELSPH                       0.003723
38) KWHSPH                         0.003689
39) ELCOOL                         0.003662
40) TOTHSQFT                       0.003649
41) KWH                            0.003643
42) DOEID                          0.003627
43) DOLNGOTH                       0.003538
44) BTUELRFG                       0.003473
45) BTUELOTH                       0.003344
46) TOTSQFT_EN                     0.003269
47) EQUIPM                         0.003265
48) DOLELOTH                       0.003239
49) BTUNGOTH                       0.003209
50) TOTALBTUOTH                    0.003190
51) TOTALDOLWTH                    0.003189
52) TOTALDOLRFG                    0.003158
53) DOLNGSPH                       0.003127
54) KWHOTH                         0.002989
55) KWHRFG                         0.002952
56) YEARMADE                       0.002924
57) BTUNGSPH                       0.002917
58) TOTALBTUWTH                    0.002892
59) CUFEETNGSPH                    0.002847
60) BTUNG                          0.002787
61) DWASHUSE                       0.002763
62) CUFEETNG                       0.002681
63) ACROOMS                        0.002605
64) PROTHERM                       0.002588
65) AGEDW                          0.002532
66) SLDDRS                         0.002484
67) TOTSQFT                        0.002449
68) USECENAC                       0.002445
69) TOTALBTURFG                    0.002428
70) USECFAN                        0.002379
71) DOLLARNG                       0.002367
72) BTUELWTH                       0.002284
73) HHAGE                          0.002281
74) EDUCATION                      0.002278
75) MAINTAC                        0.002243
76) DOOR1SUM                       0.002235
77) BASEUSE                        0.002234
78) LGT1EE                         0.002211
79) AGECENAC                       0.002187
80) DOLNGWTH                       0.002152
81) CUFEETNGWTH                    0.002149
82) METRO                          0.002139
83) HEATROOM                       0.002112
84) CELLAR                         0.002110
85) ROOFTYPE                       0.002006
86) YEARMADERANGE                  0.001999
87) BTUNGWTH                       0.001988
88) LGT1                           0.001985
89) CRAWL                          0.001946
90) CUFEETNGOTH                    0.001920
91) ACOTHERS                       0.001833
92) TOTUSQFT                       0.001813
93) PELAC                          0.001799
94) KWHWTH                         0.001749
95) NUMPC                          0.001703
96) BASECOOL                       0.001687
97) THERMAINAC                     0.001672
98) AGEHHMEMCAT2                   0.001623
99) PCPRINT                        0.001620
100) WHEATAGE                       0.001610
101) CENACHP                        0.001602
102) TEMPNITEAC                     0.001600
103) FUELHEAT                       0.001582
104) AUTOHEATNITE                   0.001547
105) TIMEON2                        0.001544
106) OVENUSE                        0.001530
107) PELHEAT                        0.001523
108) TYPEHUQ                        0.001503
109) TELLDAYS                       0.001499
110) INWIRELESS                     0.001496
111) AGECWASH                       0.001491
112) PRKGPLC2                       0.001476
113) EQUIPAGE                       0.001449
114) NUMMEAL                        0.001431
115) FUELFOOD                       0.001426
116) AGECDRYER                      0.001411
117) TIMEON1                        0.001398
118) GARGLOC                        0.001398
119) DOLELWTH                       0.001393
120) LGT4                           0.001378
121) TOTROOMS                       0.001363
122) LGT4EE                         0.001354
123) AUTOHEATDAY                    0.001323
124) AGEHHMEMCAT3                   0.001310
125) NHSLDMEM                       0.001307
126) POVERTY150                     0.001289
127) SIZEOFGARAGE                   0.001288
128) CATHCEIL                       0.001259
129) ELECDEV                        0.001232
130) PROTHERMAC                     0.001217
131) HIGHCEIL                       0.001198
132) TYPEGLASS                      0.001198
133) WINDOWS                        0.001167
134) WHEATBKT                       0.001134
135) TVSIZE2                        0.001128
136) TVCOLOR                        0.001124
137) ELFOOD                         0.001123
138) TEMPGONEAC                     0.001123
139) NUMAPTS                        0.001123
140) PRKGPLC1                       0.001117
141) DRYRFUEL                       0.001116
142) NOUTLGTNT                      0.001115
143) SIZRFRI1                       0.001113
144) TVTYPE1                        0.001100
145) MAINTHT                        0.001094
146) CORDSWD                        0.001085
147) NUMFLRS                        0.001058
148) ESCWASH                        0.001051
149) INCABLE                        0.001050
150) PCONOFF2                       0.001044
151) CABLESAT1                      0.001044
152) NUMBERAC                       0.001039
153) TVONWE1                        0.001033
154) TVONWE2                        0.001028
155) WASHLOAD                       0.001022
156) PELCOOK                        0.001014
157) CHIMNEY                        0.001014
158) STORIES                        0.001013
159) BASEHEAT                       0.001010
160) DVD2                           0.001003
161) INSATEL                        0.001002
162) FUELH2O                        0.000993
163) WELLPUMP                       0.000989
164) TYPERFR1                       0.000989
165) PCTYPE2                        0.000979
166) HELPCFL                        0.000976
167) BTUFO                          0.000967
168) OUTLET                         0.000965
169) TOPFRONT                       0.000951
170) BASEFIN                        0.000947
171) PCTYPE1                        0.000946
172) OCCUPYYRANGE                   0.000945
173) DOLLARFO                       0.000943
174) AMTMICRO                       0.000939
175) AGEHHMEMCAT4                   0.000934
176) NUMTHERM                       0.000934
177) OUTGRILLFUEL                   0.000930
178) TVONWD1                        0.000928
179) EQUIPAUX                       0.000920
180) LGTOEE                         0.000916
181) COOLTYPENOAC                   0.000914
182) CONCRETE                       0.000913
183) TVAUDIOSYS2                    0.000910
184) BATTOOLS                       0.000909
185) NCOMBATH                       0.000908
186) TYPECLN                        0.000908
187) OTHROOMS                       0.000906
188) ESFRIG                         0.000897
189) OVENFUEL                       0.000894
190) TVTYPE2                        0.000886
191) SDESCENT                       0.000884
192) AGERFRI1                       0.000877
193) PERIODNG                       0.000873
194) DRAFTY                         0.000860
195) AGEWS                          0.000853
196) STOVENFUEL                     0.000848
197) HELPHT                         0.000848
198) BATCHRG                        0.000845
199) PGASHEAT                       0.000836
200) Householder_Race               0.000825
201) WOODKILN                       0.000821
202) PCTYPE3                        0.000816
203) WASHTEMP                       0.000809
204) HELPCFLY                       0.000808
205) INVESTMT                       0.000786
206) SIZEOFDETACH                   0.000786
207) TVONWD2                        0.000783
208) CHRGPLGT                       0.000783
209) FOPAY                          0.000783
210) AGEFRZR                        0.000782
211) DIGITSTB2                      0.000776
212) SCALEFO                        0.000772
213) MICRO                          0.000770
214) BTUWOOD                        0.000768
215) OVENCLN                        0.000764
216) ELECCHRG                       0.000752
217) SIZFREEZ                       0.000736
218) RENTHELP                       0.000735
219) TELLWORK                       0.000733
220) BEDROOMS                       0.000726
221) SWIMPOOL                       0.000724
222) INTERNET                       0.000719
223) HELPWIN                        0.000717
224) AGEHHMEMCAT5                   0.000715
225) WHEATSIZ                       0.000709
226) EQMAMT                         0.000704
227) WWACAGE                        0.000702
228) ROOMHEAT                       0.000700
229) PGASHTWA                       0.000684
230) LGT12EE                        0.000680
231) HELPCW                         0.000679
232) TVONWE3                        0.000679
233) KAVALNG                        0.000678
234) INDIALUP                       0.000668
235) COMBOVCRDVD2                   0.000667
236) ESDISHW                        0.000667
237) HELPFRI                        0.000667
238) OVEN                           0.000665
239) SCALENG                        0.000664
240) TVONWDWATCH1                   0.000663
241) FPFUEL                         0.000663
242) HELPWH                         0.000657
243) CHRGPLGE                       0.000655
244) STOVEFUEL                      0.000651
245) REPLCDW                        0.000650
246) STEAMR                         0.000646
247) PCONOFF1                       0.000643
248) AGERFRI2                       0.000641
249) PLAYSTA2                       0.000637
250) INSTLCFL                       0.000630
251) CARRYEL                        0.000630
252) FOWARM                         0.000626
253) STUDIO                         0.000625
254) AGEAUD                         0.000624
255) STOVE                          0.000622
256) UGCOOK                         0.000621
257) REVERSE                        0.000620
258) HEATOTH                        0.000619
259) TIMEON3                        0.000611
260) TVONWEWATCH3                   0.000611
261) MONITOR2                       0.000609
262) RANGE                          0.000609
263) DEFROST                        0.000604
264) PUGCOOK                        0.000601
265) CWASHER                        0.000598
266) U                              0.000593
267) DRYRUSE                        0.000593
268) TVONWD3                        0.000591
269) UPRTFRZR                       0.000590
270) TVONWEWATCH1                   0.000590
271) FREEZER                        0.000588
272) NEWGLASS                       0.000586
273) DVR2                           0.000585
274) DISHWASH                       0.000580
275) LPGDELV                        0.000579
276) COMBODVR1                      0.000578
277) PUGOTH                         0.000577
278) CABLESAT2                      0.000573
279) DVR1                           0.000573
280) STEREO                         0.000573
281) AGEINS                         0.000572
282) PERIODEL                       0.000572
283) ZEQUIPAGE                      0.000571
284) TVTYPE3                        0.000563
285) WDPELLET                       0.000558
286) ATTIC                          0.000558
287) WDOTHER                        0.000557
288) DIGITSTB1                      0.000552
289) INDSL                          0.000552
290) SIZRFRI2                       0.000549
291) TOASTER                        0.000548
292) ZTEMPGONE                      0.000544
293) HELPWS                         0.000542
294) COMBODVR2                      0.000539
295) ANSMACH                        0.000533
296) PLAYSTA3                       0.000529
297) ADQINSUL                       0.000527
298) ICE                            0.000521
299) TVSIZE1                        0.000516
300) ELWATER                        0.000514
301) TYPERFR2                       0.000512
302) USEWWAC                        0.000510
303) HELPDWY                        0.000505
304) USENG                          0.000502
305) VCR2                           0.000500
306) PCSLEEP1                       0.000498
307) CABLESAT3                      0.000496
308) WOODAMT                        0.000496
309) PERMELEC                       0.000493
310) NAPTFLRS                       0.000491
311) REFRIGT2                       0.000490
312) KOWNRENT                       0.000485
313) NUMFRIG                        0.000484
314) HELPHTY                        0.000483
315) REPLCFRI                       0.000482
316) GARGHEAT                       0.000481
317) HHSEX                          0.000481
318) BTULPOTH                       0.000477
319) TEMPHOMEAC                     0.000474
320) USEMOISTURE                    0.000473
321) PERIODFO                       0.000466
322) PCONOFF3                       0.000466
323) KAVALEL                        0.000466
324) ATTICFIN                       0.000458
325) NONE                           0.000454
326) INSTLWS                        0.000453
327) DVD3                           0.000451
328) COFFEE                         0.000451
329) ATTCCOOL                       0.000449
330) WORKPAY                        0.000446
331) USENGFP                        0.000444
332) NOCORD                         0.000440
333) DVD1                           0.000437
334) VCR3                           0.000435
335) REPLCCW                        0.000435
336) DNTHEAT                        0.000429
337) NGFPFLUE                       0.000427
338) HELPDW                         0.000426
339) TVSIZE3                        0.000426
340) TVONWDWATCH3                   0.000426
341) HUPROJ                         0.000426
342) MONITOR1                       0.000423
343) FUELTUB                        0.000423
344) VCR1                           0.000422
345) WARMAIR                        0.000421
346) WDSCRAP                        0.000420
347) BTULP                          0.000419
348) INSTLINS                       0.000416
349) RNSETEMP                       0.000416
350) COMBOVCRDVD1                   0.000416
351) USELP                          0.000412
352) ELECAUX                        0.000409
353) WOODLOGS                       0.000407
354) DOLFOSPH                       0.000404
355) FOODASST                       0.000400
356) AQUARIUM                       0.000399
357) AUDIT                          0.000399
358) MONRFRI2                       0.000398
359) POVERTY100                     0.000397
360) WOODAUX                        0.000393
361) TVAUDIOSYS3                    0.000391
362) OTHERSTB2                      0.000388
363) TVAUDIOSYS1                    0.000388
364) HELPCWY                        0.000387
365) ESFRIG2                        0.000387
366) RGLRPAY                        0.000383
367) RETIREPY                       0.000381
368) ZWHEATBKT                      0.000379
369) ZHouseholder_Race              0.000378
370) OTHERSTB1                      0.000376
371) THERMAIN                       0.000376
372) ATTCHEAT                       0.000376
373) PIPELESS                       0.000371
374) UGWATER                        0.000369
375) FURNFUEL                       0.000367
376) HELPWINY                       0.000366
377) DRYER                          0.000366
378) PERIODLP                       0.000365
379) LPGPAY                         0.000364
380) REPLCHT                        0.000363
381) AUTOCOOLNITE                   0.000363
382) GALLONLPSPH                    0.000361
383) COMBODVR3                      0.000361
384) ZTEMPNITE                      0.000361
385) LGT12                          0.000360
386) PLAYSTA1                       0.000359
387) AGEHHMEMCAT7                   0.000357
388) HELPFRIY                       0.000357
389) ZYEARMADE                      0.000353
390) ZCONDCOOP                      0.000353
391) ZMONEYPY                       0.000350
392) ZWHEATAGE                      0.000349
393) ZWHEATSIZ                      0.000340
394) GALLONFO                       0.000338
395) CONDCOOP                       0.000337
396) DIGITSTB3                      0.000335
397) WHEATOTH                       0.000335
398) ZCONCRETE                      0.000335
399) EQUIPNOHEAT                    0.000331
400) GARGCOOL                       0.000329
401) DOLLARLP                       0.000327
402) NUMFREEZ                       0.000327
403) AUTOCOOLDAY                    0.000325
404) HELPCAC                        0.000324
405) NHAFBATH                       0.000323
406) HSFUEL                         0.000322
407) ZTOTUCSQFT                     0.000321
408) TVONWDWATCH2                   0.000321
409) PCSLEEP2                       0.000320
410) SCALEEL                        0.000319
411) LPOTHER                        0.000315
412) PELHOTWA                       0.000314
413) TVONWEWATCH2                   0.000311
414) ESWWAC                         0.000308
415) ZTYPECLN                       0.000306
416) HELPWWAC                       0.000306
417) DVR3                           0.000306
418) USESOLAR                       0.000302
419) ZTOTSQFT                       0.000300
420) NGASLIGHT                      0.000298
421) EMPLOYHH                       0.000297
422) R                              0.000296
423) CONVERSION                     0.000295
424) BASEHT2                        0.000294
425) AGEHHMEMCAT6                   0.000293
426) ELWARM                         0.000291
427) ZOVENCLN                       0.000290
428) ATTICUSE                       0.000288
429) UGOTH                          0.000288
430) USEWOOD                        0.000286
431) ATHOME                         0.000284
432) STOVEN                         0.000280
433) COPIER                         0.000278
434) HBUSNESS                       0.000277
435) ZYEARMADERANGE                 0.000275
436) PCSLEEP3                       0.000274
437) REFRIGT1                       0.000272
438) TREESHAD                       0.000271
439) FINBASERMS                     0.000268
440) DOLLPOTH                       0.000268
441) SPOUSE                         0.000268
442) REPLCFRZ                       0.000264
443) COMPUTER                       0.000263
444) TYPEHUQ4                       0.000263
445) WDWARM                         0.000262
446) GALLONLPOTH                    0.000256
447) ZTVTYPE1                       0.000255
448) ZAGEDW                         0.000252
449) HELPINS                        0.000248
450) WDOTHUSE                       0.000244
451) ZPGASHTWA                      0.000236
452) RECBATH                        0.000232
453) USEOTH                         0.000231
454) UGASAUX                        0.000229
455) COMBOVCRDVD3                   0.000228
456) HELPFRZ                        0.000228
457) ZTOTCSQFT                      0.000225
458) ZTOTHSQFT                      0.000222
459) SOLOTHER                       0.000222
460) GALLONFOSPH                    0.000221
461) GALLONLPWTH                    0.000219
462) POOL                           0.000219
463) ZNUMAPTS                       0.000216
464) ZTOTUSQFT                      0.000208
465) OUTGRILL                       0.000206
466) KFUELOT                        0.000205
467) ZTOTSQFT_EN                    0.000204
468) FUELH2O2                       0.000203
469) BTUFOSPH                       0.000203
470) ZAGECDRYER                     0.000202
471) DIFEQUIP                       0.000202
472) OTHWORK                        0.000201
473) BTUFOWTH                       0.000199
474) SEPFREEZ                       0.000199
475) HEATHOME                       0.000198
476) ZFUELH2O                       0.000196
477) LPCOOK                         0.000194
478) SIZFREEZ2                      0.000193
479) ZREFRIGT1                      0.000191
480) ZTYPEGLASS                     0.000191
481) MONITOR3                       0.000191
482) OTHERSTB3                      0.000189
483) ONSITEGRID                     0.000186
484) FAX                            0.000186
485) ZWASHTEMP                      0.000179
486) SSINCOME                       0.000178
487) USEFO                          0.000177
488) ZTEMPGONEAC                    0.000175
489) USENOTMOIST                    0.000172
490) MOISTURE                       0.000171
491) ZWHEATOTH                      0.000170
492) DOLLPWTH                       0.000168
493) DOLLPSPH                       0.000167
494) UGWARM                         0.000167
495) REPLCWWAC                      0.000166
496) STGRILA                        0.000166
497) AGEHHMEMCAT8                   0.000165
498) PELLIGHT                       0.000164
499) ZRETIREPY                      0.000161
500) ZWALLTYPE                      0.000159
501) RADFUEL                        0.000157
502) RMHTFUEL                       0.000156
503) ZEQUIPM                        0.000155
504) GALLONLP                       0.000152
505) HELPAUD                        0.000148
506) ZTEMPHOME                      0.000146
507) REPLCCAC                       0.000144
508) ZAGERFRI1                      0.000142
509) MICRO                          0.000140
510) HELPAUDY                       0.000137
511) LPWARM                         0.000137
512) SCALELP                        0.000135
513) UPRTFRZR2                      0.000131
514) PERIODKR                       0.000129
515) ZCENACHP                       0.000128
516) ZATTIC                         0.000125
517) ZCOMBODVR1                     0.000125
518) DOLLARKER                      0.000124
519) H2OTYPE2                       0.000124
520) ZCRAWL                         0.000124
521) ZPGASHEAT                      0.000123
522) LPWATER                        0.000123
523) ZAGECWASH                      0.000122
524) AGERFRI3                       0.000121
525) ZEDUCATION                     0.000120
526) NUMH2OHTRS                     0.000117
527) ZHUPROJ                        0.000117
528) HELPFRZY                       0.000116
529) ZHHAGE                         0.000112
530) H2OTYPE1                       0.000111
531) ZTVONWEWATCH1                  0.000108
532) ZSIZRFRI1                      0.000108
533) ZINSTLWS                       0.000107
534) ZDVR1                          0.000106
535) HELPCACY                       0.000102
536) ZAGECENAC                      0.000102
537) FREEZER2                       0.000101
538) ZOVENUSE                       0.000101
539) SIZRFRI3                       0.000099
540) ZINDIALUP                      0.000096
541) RNGFUEL                        0.000095
542) ZCASHBEN                       0.000094
543) ZFUELFOOD                      0.000094
544) FUELNOHEAT                     0.000093
545) FINATTRMS                      0.000092
546) ONSITE                         0.000092
547) ZFUELHEAT                      0.000090
548) GALLONKER                      0.000087
549) WHEATAGE2                      0.000086
550) ZPELHOTWA                      0.000084
551) CARRYKER                       0.000083
552) ZWELLPUMP                      0.000082
553) ESFRIG3                        0.000081
554) ZMAINTAC                       0.000081
555) ZINCABLE                       0.000081
556) TYPERFR3                       0.000080
557) DOLFOWTH                       0.000077
558) BTUKERSPH                      0.000077
559) ZINSATEL                       0.000075
560) PCTBSTHT                       0.000075
561) ZCOMBOVCRDVD3                  0.000074
562) AGEHHMEMCAT12                  0.000073
563) MONRFRI3                       0.000072
564) NOTMOIST                       0.000072
565) DOLKERSPH                      0.000070
566) NUMH2ONOTNK                    0.000068
567) ZMAINTHT                       0.000067
568) ZINVESTMT                      0.000066
569) ZNUMH2ONOTNK                   0.000066
570) ZWORKPAY                       0.000064
571) ZNUMTHERM                      0.000063
572) ZTVONWE1                       0.000062
573) ZOTHERSTB2                     0.000062
574) ZINWIRELESS                    0.000061
575) OTHERWAYEL                     0.000060
576) ZWWACAGE                       0.000060
577) FUELPOOL                       0.000059
578) GALLONKEROTH                   0.000059
579) BTULPWTH                       0.000058
580) KERODEL                        0.000058
581) BTULPSPH                       0.000058
582) SCALEKER                       0.000056
583) AGEHHMEMCAT9                   0.000056
584) KEROCASH                       0.000055
585) CASHBEN                        0.000055
586) ZLPGPAY                        0.000055
587) ZUSEWWAC                       0.000054
588) WHEATSIZ2                      0.000053
589) ZDIGITSTB1                     0.000053
590) BASECL2                        0.000053
591) ZROOFTYPE                      0.000052
592) OTHERWAYNG                     0.000052
593) ZAGEHHMEMCAT2                  0.000052
594) ZATHOME                        0.000051
595) ZDVR2                          0.000051
596) DOLKEROTH                      0.000051
597) ZAUTOHEATNITE                  0.000049
598) SOLWATER                       0.000049
599) ZRNSETEMP                      0.000048
600) ZTVONWEWATCH3                  0.000047
601) ZAGEWS                         0.000045
602) AGEHHMEMCAT10                  0.000043
603) ZTVONWDWATCH3                  0.000042
604) ZAUDIT                         0.000042
605) BTUKEROTH                      0.000041
606) ZELECDEV                       0.000041
607) ORIG1FAM                       0.000041
608) HELPWHY                        0.000040
609) ZTVTYPE2                       0.000040
610) ZUSECFAN                       0.000039
611) LOOKLIKE                       0.000039
612) TOPGRILL                       0.000039
613) HELPINSY                       0.000039
614) ZAGERFRI2                      0.000039
615) ZFOODASST                      0.000038
616) SOLARAUX                       0.000036
617) HELPWSY                        0.000036
618) AGEFRZR2                       0.000036
619) AGEHHMEMCAT11                  0.000036
620) ZCOMBODVR3                     0.000036
621) REFRIGT3                       0.000035
622) ZWHEATAGE2                     0.000035
623) ZAGEFRZR                       0.000033
624) FOOTHER                        0.000032
625) DUCTS                          0.000032
626) ZFURNFUEL                      0.000032
627) ZICE                           0.000030
628) ZNGFPFLUE                      0.000030
629) NKRGALNC                       0.000029
630) ZDEFROST                       0.000028
631) ZHBUSNESS                      0.000028
632) ZRENTHELP                      0.000028
633) ZPELHEAT                       0.000028
634) ZOTHWORK                       0.000028
635) ZPCONOFF1                      0.000028
636) ZNHSLDMEM                      0.000027
637) ZSWIMPOOL                      0.000027
638) ZWOODAMT                       0.000023
639) ZRGLRPAY                       0.000023
640) ATTCHT2                        0.000023
641) ZREFRIGT2                      0.000021
642) ZTHERMAIN                      0.000021
643) LPGAUX                         0.000021
644) ZINDSL                         0.000021
645) ZLOOKLIKE                      0.000020
646) ZMONITOR1                      0.000020
647) ZATTICFIN                      0.000020
648) ZACROOMS                       0.000019
649) OTHWARM                        0.000019
650) ZTHERMAINAC                    0.000019
651) ZTVONWD1                       0.000016
652) OTHERWAYLPG                    0.000016
653) ZTYPEHUQ4                      0.000015
654) USEKERO                        0.000015
655) ZWHEATSIZ2                     0.000015
656) ZGARGCOOL                      0.000015
657) ZPCSLEEP1                      0.000015
658) ZPRKGPLC1                      0.000015
659) ZTEMPHOMEAC                    0.000015
660) ZTVAUDIOSYS2                   0.000015
661) PIPEFUEL                       0.000015
662) ZPLAYSTA2                      0.000015
663) ZGARGLOC                       0.000015
664) ZNUMFRIG                       0.000014
665) AGEHHMEMCAT13                  0.000014
666) ZORIG1FAM                      0.000014
667) OTHERAUX                       0.000014
668) ZCONVERSION                    0.000014
669) ZEQMAMT                        0.000014
670) ZDNTAC                         0.000014
671) ZPROTHERMAC                    0.000014
672) ZCHRGPLGT                      0.000014
673) ZAIRCOND                       0.000013
674) ZTELLWORK                      0.000013
675) ZRADFUEL                       0.000013
676) ZEMPLOYHH                      0.000013
677) ZOTHROOMS                      0.000013
678) ZTIMEON2                       0.000013
679) ZWDOTHER                       0.000012
680) ZSSINCOME                      0.000012
681) ZNUMH2OHTRS                    0.000012
682) ZFUELNOHEAT                    0.000011
683) ZINTERNET                      0.000011
684) ZAGEHHMEMCAT6                  0.000011
685) ZATTCCOOL                      0.000011
686) ZLGT1                          0.000010
687) GALLONKERSPH                   0.000009
688) ZDUCTS                         0.000009
689) ZAGEHHMEMCAT3                  0.000008
690) ZPRKGPLC2                      0.000008
691) OTHCOOK                        0.000007
692) ZLGT12                         0.000007
693) NUMCORDS                       0.000006
694) BTUKER                         0.000006
695) ZAUTOCOOLDAY                   0.000004
696) ZTEMPNITEAC                    0.000004
697) ZSPOUSE                        0.000003
698) NOCRCASH                       0.000003
699) ZSDESCENT                      0.000002
700) ZTELLDAYS                      0.000000
701) ZNUMCFAN                       0.000000
702) ELOTHER                        0.000000
703) BTUFOOTH                       0.000000
704) KEROAUX                        0.000000
705) ZUSENOTMOIST                   0.000000
706) ZTREESHAD                      0.000000
707) ZNOTMOIST                      0.000000
708) KRWATER                        0.000000
709) USEEL                          0.000000
710) SOLWARM                        0.000000
711) ZAGEAUD                        0.000000
712) WDWATER                        0.000000
713) KROTHER                        0.000000
714) GALLONFOOTH                    0.000000
715) ZLGT4                          0.000000
716) GALLONFOWTH                    0.000000
717) ZPCTBSTHT                      0.000000
718) ZBASEHEAT                      0.000000
719) ZFINBASERMS                    0.000000
720) ZBASEFIN                       0.000000
721) ZCELLAR                        0.000000
722) ZNHAFBATH                      0.000000
723) ZNCOMBATH                      0.000000
724) ZBEDROOMS                      0.000000
725) ZSTORIES                       0.000000
726) GALLONKERWTH                   0.000000
727) ZNAPTFLRS                      0.000000
728) ZSTUDIO                        0.000000
729) ZNUMFLRS                       0.000000
730) ZOCCUPYYRANGE                  0.000000
731) PCTBSTCL                       0.000000
732) ZHIGHCEIL                      0.000000
733) ZCATHCEIL                      0.000000
734) ZKOWNRENT                      0.000000
735) ZPOOL                          0.000000
736) BTUKERWTH                      0.000000
737) ZFUELPOOL                      0.000000
738) PCTATTCL                       0.000000
739) ATTCCL2                        0.000000
740) ZRECBATH                       0.000000
741) PCTATTHT                       0.000000
742) ZFUELTUB                       0.000000
743) ZBASEHT2                       0.000000
744) ZBASECOOL                      0.000000
745) KRWARM                         0.000000
746) ZBASECL2                       0.000000
747) FOWATER                        0.000000
748) FOILAUX                        0.000000
749) DOLKERWTH                      0.000000
750) ZSIZEOFDETACH                  0.000000
751) ZDRAFTY                        0.000000
752) ZAGEINS                        0.000000
753) ZGARGHEAT                      0.000000
754) ZINSTLINS                      0.000000
755) ZADQINSUL                      0.000000
756) ZNEWGLASS                      0.000000
757) ZSIZEOFGARAGE                  0.000000
758) ZATTICUSE                      0.000000
759) ZWINDOWS                       0.000000
760) ZDOOR1SUM                      0.000000
761) ZATTCCL2                       0.000000
762) ZPCTATTCL                      0.000000
763) ZSLDDRS                        0.000000
764) ZPCTATTHT                      0.000000
765) ZATTCHT2                       0.000000
766) ZATTCHEAT                      0.000000
767) ZNGASLIGHT                     0.000000
768) ZNUMBERAC                      0.000000
769) ZNOUTLGTNT                     0.000000
770) ZBASEUSE                       0.000000
771) ZPCTBSTCL                      0.000000
772) ZFINATTRMS                     0.000000
773) ZONSITE                        0.000000
774) ZAGEHHMEMCAT14                 0.000000
775) ZAGEHHMEMCAT13                 0.000000
776) ZTVSIZE3                       0.000000
777) ZTVTYPE3                       0.000000
778) ZCABLESAT3                     0.000000
779) ZDVR3                          0.000000
780) ZDIGITSTB3                     0.000000
781) ZPLAYSTA3                      0.000000
782) ZVCR3                          0.000000
783) ZDVD3                          0.000000
784) ZTVAUDIOSYS3                   0.000000
785) ZOTHERSTB3                     0.000000
786) ZTVONWD3                       0.000000
787) ZTVONWE3                       0.000000
788) ZCOMPUTER                      0.000000
789) ZNUMPC                         0.000000
790) ZPCTYPE1                       0.000000
791) ZTIMEON1                       0.000000
792) ZPCTYPE2                       0.000000
793) ZTVONWEWATCH2                  0.000000
794) ZTVONWE2                       0.000000
795) ZTVONWDWATCH2                  0.000000
796) ZOTHERSTB1                     0.000000
797) ZRMHTFUEL                      0.000000
798) ZCABLESAT1                     0.000000
799) ZPLAYSTA1                      0.000000
800) ZCOMBOVCRDVD1                  0.000000
801) ZVCR1                          0.000000
802) ZDVD1                          0.000000
803) ZTVAUDIOSYS1                   0.000000
804) ZTVONWDWATCH1                  0.000000
805) ZTVONWD2                       0.000000
806) ZTVSIZE2                       0.000000
807) ZCABLESAT2                     0.000000
808) ZCOMBODVR2                     0.000000
809) ZDIGITSTB2                     0.000000
810) ZCOMBOVCRDVD2                  0.000000
811) ZVCR2                          0.000000
812) ZDVD2                          0.000000
813) ZMONITOR2                      0.000000
814) ZPCONOFF2                      0.000000
815) ZPCSLEEP2                      0.000000
816) ZEQUIPNOHEAT                   0.000000
817) ZOTHERWAYLPG                   0.000000
818) ZOTHERWAYFO                    0.000000
819) ZHEATOTH                       0.000000
820) ZFOPAY                         0.000000
821) ZOTHERWAYNG                    0.000000
822) ZPUGOTH                        0.000000
823) ZPUGCOOK                       0.000000
824) DOLFOOTH                       0.000000
825) ZKEROCASH                      0.000000
826) ZDNTHEAT                       0.000000
827) ZOTHERWAYEL                    0.000000
828) ZPELLIGHT                      0.000000
829) ZHEATHOME                      0.000000
830) ZPELAC                         0.000000
831) DIFFUEL                        0.000000
832) ZPELCOOK                       0.000000
833) ZKERODEL                       0.000000
834) ZPIPEFUEL                      0.000000
835) ZPCTYPE3                       0.000000
836) ZDIPSTICK                      0.000000
837) ZMONITOR3                      0.000000
838) ZTIMEON3                       0.000000
839) ZPCONOFF3                      0.000000
840) ZPCSLEEP3                      0.000000
841) ZPCPRINT                       0.000000
842) ZFAX                           0.000000
843) ZCOPIER                        0.000000
844) ZSWAMPCOL                      0.000000
845) ZCHRGPLGE                      0.000000
846) ZAQUARIUM                      0.000000
847) ZSTEREO                        0.000000
848) ZNOCORD                        0.000000
849) ZANSMACH                       0.000000
850) ZBATTOOLS                      0.000000
851) ZBATCHRG                       0.000000
852) ZELECCHRG                      0.000000
853) ZTVSIZE1                       0.000000
854) ZTVCOLOR                       0.000000
855) ZHSFUEL                        0.000000
856) ZUPRTFRZR                      0.000000
857) ZTYPERFR3                      0.000000
858) ZSIZRFRI3                      0.000000
859) ZREFRIGT3                      0.000000
860) ZMONRFRI3                      0.000000
861) ZAGERFRI3                      0.000000
862) ZSEPFREEZ                      0.000000
863) ZNUMFREEZ                      0.000000
864) ZSIZFREEZ                      0.000000
865) ZSIZRFRI2                      0.000000
866) ZFREEZER                       0.000000
867) ZUPRTFRZR2                     0.000000
868) ZSIZFREEZ2                     0.000000
869) ZFREEZER2                      0.000000
870) ZAGEFRZR2                      0.000000
871) ZDISHWASH                      0.000000
872) ZDWASHUSE                      0.000000
873) ZMONRFRI2                      0.000000
874) ZTYPERFR2                      0.000000
875) ZAGEHHMEMCAT11                 0.000000
876) ZMICRO                         0.000000
877) ZSTOVEN                        0.000000
878) ZSTOVENFUEL                    0.000000
879) ZSTOVE                         0.000000
880) ZSTOVEFUEL                     0.000000
881) ZOVEN                          0.000000
882) ZOVENFUEL                      0.000000
883) ZAUTOCOOLNITE                  0.000000
884) ZAMTMICRO                      0.000000
885) ZTYPERFR1                      0.000000
886) ZOUTGRILL                      0.000000
887) ZOUTGRILLFUEL                  0.000000
888) ZTOPGRILL                      0.000000
889) ZSTGRILA                       0.000000
890) ZTOASTER                       0.000000
891) ZNUMMEAL                       0.000000
892) ZCOFFEE                        0.000000
893) ZAGEHHMEMCAT12                 0.000000
894) ZAGEHHMEMCAT10                 0.000000
895) ZDRYRUSE                       0.000000
896) ZWDSCRAP                       0.000000
897) ZHEATROOM                      0.000000
898) ZDIFFUEL                       0.000000
899) ZNUMCORDS                      0.000000
900) ZRNGFUEL                       0.000000
901) ZUSENGFP                       0.000000
902) ZFPFUEL                        0.000000
903) ZWDPELLET                      0.000000
904) ZWOODLOGS                      0.000000
905) ZAUTOHEATDAY                   0.000000
906) ZNKRGALNC                      0.000000
907) ZNOCRCASH                      0.000000
908) ZCWASHER                       0.000000
909) ZTOPFRONT                      0.000000
910) ZWASHLOAD                      0.000000
911) ZDRYER                         0.000000
912) ZDRYRFUEL                      0.000000
913) ZPROTHERM                      0.000000
914) ZMOISTURE                      0.000000
915) ZAGEHHMEMCAT9                  0.000000
916) ZCOOLTYPENOAC                  0.000000
917) ZAGEHHMEMCAT8                  0.000000
918) ZAGEHHMEMCAT7                  0.000000
919) ZONSITEGRID                    0.000000
920) ZAGEHHMEMCAT5                  0.000000
921) ZAGEHHMEMCAT4                  0.000000
922) ZACOTHERS                      0.000000
923) ZCOOLTYPE                      0.000000
924) OTHWATER                       0.000000
925) ZUSEMOISTURE                   0.000000
926) HELPWWACY                      0.000000
927) ZHHSEX                         0.000000
928) OTHERWAYFO                     0.000000
929) ZFUELH2O2                      0.000000
930) ZH2OTYPE2                      0.000000
931) ZH2OTYPE1                      0.000000
932) AGEHHMEMCAT14                  0.000000
933) ZUSECENAC                      0.000000
"""

In [127]:
months = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']
days = ['mon','tue','wed','thu','fri','sat','sun']

def get_month_number(month_name):
    return months.index(month_name)

def get_day_number(day_name):
    return days.index(day_name)

X_fires_data = pandas.read_csv('forestfires.csv', ',')  # load rows from CSV file

y_fires_data = X_fires_data['area']
X_fires_data.drop(columns=['area'], inplace=True)
X_fires_data['month'] = X_fires_data['month'].apply(get_month_number) # transform months to integers from 0 to 11
X_fires_data['day'] = X_fires_data['day'].apply(get_day_number) # transform days to integers from 0 to 6
X_fires_data.drop(columns=['month', 'rain'], inplace=True)

X_fires_train, X_fires_test, y_fires_train, y_fires_test = train_test_split(X_fires_data, y_fires_data, test_size=TEST_SIZE, random_state=0)
X_fires_train_scaled = scaler.fit_transform(X_fires_train, y_fires_train)
X_fires_test_scaled = scaler.fit_transform(X_fires_test, y_fires_test)
#y_fires_train_scaled = scaler.fit_transform(y_fires_train)
#y_fires_test_scaled = scaler.fit_transform(y_fires_test)

In [ ]:
"""
Forest fires importances:

 1) temp                           0.416319
 2) DMC                            0.132569
 3) RH                             0.088106
 4) ISI                            0.075632
 5) FFMC                           0.069701
 6) wind                           0.056796
 7) DC                             0.055511
 8) Y                              0.044505
 9) day                            0.027999
10) X                              0.026433
11) month                          0.006413
12) rain                           0.000017
"""

In [128]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(random_state=1, hidden_layer_sizes=(50), max_iter=180, activation='relu', solver='adam', learning_rate='constant')
classifier.fit(X_houses_train_scaled, y_houses_train)
y_pred = classifier.predict(X_houses_test_scaled)

cm = confusion_matrix(y_true=y_houses_test, y_pred=y_pred)
print('Accuracy of MLPClassifier : \n', cm)
print("\nClassification Report of MLPClassifier:")
print(classification_report(y_houses_test, y_pred))

Accuracy of MLPClassifier : 
 [[ 113    3]
 [   4 2272]]

Classification Report of MLPClassifier:
              precision    recall  f1-score   support

           A       0.97      0.97      0.97       116
           B       1.00      1.00      1.00      2276

    accuracy                           1.00      2392
   macro avg       0.98      0.99      0.98      2392
weighted avg       1.00      1.00      1.00      2392



C:\Users\Nikita\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (180) reached and the optimization hasn't converged yet.
  warnings.warn(


In [124]:
R2_ACCURACY = 3
MSE_ACCURACY = 2
MAE_ACCURACY = 2

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor

regressor = MLPRegressor(random_state=1, hidden_layer_sizes=(4), activation='identity', solver='lbfgs', max_iter=20, learning_rate='constant')
regressor.fit(X_fires_train_scaled, y_fires_train)
y_pred = regressor.predict(X_fires_test_scaled)

print('R2_score: ', round(r2_score(y_fires_test, y_pred), R2_ACCURACY))
print('MSE: ', round(mean_squared_error(y_fires_test, y_pred), MSE_ACCURACY))
print('MAE: ', round(mean_absolute_error(y_fires_test, y_pred), MAE_ACCURACY))
                   

R2_score:  -0.001
MSE:  12104.1
MAE:  25.92


In [125]:
# Отчёт: 
#
# Для MLPClassifier на наборе данных "Houses" предпочтительными являются параметры hidden_layer_sizes=(50), max_iter=180, activation='relu', solver='adam',
# learning_rate='constant'. При изменении значений  max_iter и hidden_layer_sizes, точность классификации снижается.
# Таким образом, для выбранной задачи бинарной классификации MLPClassifier работает немного менее точнее, чем GradientBoostingClassifier, и немного точнее,
# чем RandomForestClassifier.
#
# Для MLPRegressor на наборе данных "Лесные пожары" предпочтительными являются параметры activation='identity', solver='lbfgs', max_iter=20,
# learning_rate='constant', hidden_layer_sizes=(4). При увеличении max_iter, количества нейронов в слое или количества слоёв, точность существенно
# не увеличивается. Таким образом, в сревнении с RandomForestRegressor, MLPRegressor имеет близкие значения R2_score и MAE, но значительно худшее значение MSE.